In [205]:
import pandas as pd
import numpy as np

In [206]:
##############
#   Part I   #
##############

In [219]:
# http://www3.cde.ca.gov/researchfiles/sarc/sarc1516/schdetail.txt
publicschools = pd.read_table("publicschools.txt") 
publicschools.head()

# http://www.cde.ca.gov/ds/si/ps/
privateschools = pd.read_csv("privateschools1516.csv")
privateschools[:2]

,Affidavit ID,County,CDS Code,School,Street,City,State,Zip,Mailing Street,Mailing City,...,Religious Denomination,Director/Principal Officer Salutation,Director/Principal Officer First Name,Director/Principal Officer Last Name,Director/Principal Officer Title,Director/Principal Officer Email,Custodian of Records Salutation,Custodian of Records First Name,Custodian of Records Last Name,Custodian of Records Email
0,171089,Alameda,01612596910343,Agnes Memorial Christian Academey,2372 International Boulevard,Oakland,CA,94601,NaN,NaN,...,NaN,Rev.,L,Simmons,Superintendent,NaN,Rev.,M,Simmons,NaN
1,175255,Alameda,01611926965909,All Saints Catholic School,22870 Second Street,Hayward,CA,94541,NaN,NaN,...,Roman Catholic,Mrs.,Jennifer,Diaz,Principal,jdiaz@csdo.org,Ms.,Susan,Porter,sporter@csdo.org


In [208]:
# Get a dataset of CDS Codes of all public and private schools

publics = publicschools[["CDSCode", "School", "CITY"]]
publics["Type"] = "Public"
privates = pd.DataFrame(privateschools[["CDS Code", "School ", "City"]])
privates.columns = ["CDSCode", "School", "CITY"]
privates["Type"] = "Private"

schools = pd.concat([publics, privates])
schools.head()

/Library/Python/2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,CDSCode,School,CITY,Type
0,1611430131177,Berkeley High,Berkeley,Public
1,1611436056857,Martin Luther King Middle,Berkeley,Public
2,1611436056865,Willard Middle,Berkeley,Public
3,1611436090187,Rosa Parks Environmental Science Magnet,Berkeley,Public
4,1611436090195,Cragmont Elementary,Berkeley,Public


In [209]:
# http://www.cde.ca.gov/ta/ac/sa/accessdata1516.asp
# CAASPP Results 15-16
# Percentage of Students Meeting or Exceeding the State Standards

caaspp = pd.read_csv("CAASPPTestResults.csv")

### ### ### ### ###

# http://www.cde.ca.gov/ds/sd/sd/filescohort.asp
# Graduation rate 14-15

grad_rate = pd.read_table('filescohort15.txt')
# We don't care about breakdowns
grad_rate = grad_rate[(grad_rate["Subgroup"] == "All") & (grad_rate["Subgrouptype"] == "All")] 
grad_rate = grad_rate[["CDS", "Cohort Graduation Rate"]]

In [210]:
schools[schools["School"] == "Polytechnic School"]

,CDSCode,School,CITY,Type
984,19648816935589,Polytechnic School,Pasadena,Private


In [211]:
schools_caaspp = pd.merge(schools, caaspp, how='left', right_on="CDS", left_on="CDSCode")
schools_caaspp = schools_caaspp[["CDSCode", "School", "CITY", "SELA_Y2", "SMATH_Y2", "Type"]]
schools_caaspp_grad = pd.merge(schools_caaspp, grad_rate, how='left', right_on="CDS", left_on="CDSCode")
schools_caaspp_grad = schools_caaspp_grad[["CDSCode", "School", "CITY", "SELA_Y2", "SMATH_Y2", "Type", "Cohort Graduation Rate"]]
schools_caaspp_grad.columns = ["CDS", "School", "City", "ELA", "Math", "Type", "Cohort Graduation Rate"]
schools_caaspp_grad.head()

,CDS,School,City,ELA,Math,Type,Cohort Graduation Rate
0,1611430131177,Berkeley High,Berkeley,70,50,Public,87.5
1,1611436056857,Martin Luther King Middle,Berkeley,67,66,Public,NaN
2,1611436056865,Willard Middle,Berkeley,59,48,Public,NaN
3,1611436090187,Rosa Parks Environmental Science Magnet,Berkeley,56,63,Public,NaN
4,1611436090195,Cragmont Elementary,Berkeley,68,65,Public,NaN


In [212]:
# Now separate into two groups: 
# Nearby school districts, and Pasadena schools
# We can't get information for private schools, so we'll compare Pasadena publics to non-Pas publics

hoods = ["South Pasadena", "Arcadia", "San Marino", "La Canada", "Sierra Madre"]
other_schools = schools_caaspp_grad[schools_caaspp_grad["City"].isin(hoods)]
other_schools

,CDS,School,City,ELA,Math,Type,Cohort Graduation Rate
1462,19642611930288,Arcadia High,Arcadia,77,73,Public,98.71
1463,19642611995299,Rancho Learning Center (Alternative),Arcadia,48,16,Public,70.73
1464,19642616011134,Baldwin Stocker Elementary,Arcadia,77,82,Public,NaN
1465,19642616011159,Camino Grove Elementary,Arcadia,69,71,Public,NaN
1466,19642616011167,Highland Oaks Elementary,Arcadia,81,79,Public,NaN
1467,19642616011175,Holly Avenue Elementary,Arcadia,72,74,Public,NaN
1468,19642616011183,Hugo Reid Elementary,Arcadia,79,75,Public,NaN
1469,19642616011191,Longley Way Elementary,Arcadia,69,71,Public,NaN
1470,19642616057467,First Avenue Middle,Arcadia,75,73,Public,NaN
1471,19642616057475,Richard Henry Dana Middle,Arcadia,76,82,Public,NaN


In [213]:
PUSD = ["Pasadena", "Altadena"]
pasadena_schools = schools_caaspp_grad[schools_caaspp_grad["City"].isin(PUSD)]
pasadena_schools

,CDS,School,City,ELA,Math,Type,Cohort Graduation Rate
2691,19648811931062,Blair High,Pasadena,43,30,Public,73.43
2692,19648811931674,Marshall Fundamental,Pasadena,54,37,Public,96.25
2693,19648811932409,Norma Coombs Elementary,Pasadena,51,46,Public,NaN
2694,19648811936103,John Muir High,Pasadena,42,16,Public,86.47
2695,19648811936806,Rose City High (Continuation),Pasadena,NaN,NaN,Public,81.46
2696,19648811936822,Pasadena High,Pasadena,57,33,Public,93.13
2697,19648816021505,Altadena Elementary,Altadena,19,12,Public,NaN
2698,19648816021547,Cleveland Elementary,Pasadena,19,13,Public,NaN
2699,19648816021554,Don Benito Fundamental,Pasadena,58,48,Public,NaN
2700,19648816021570,Field (Eugene) Elementary,Pasadena,62,56,Public,NaN


In [214]:
###############
#   Part II   #
###############

In [215]:
# Pasadena has so many private schools! Let's compare it to counts for public schools.
privateschools_enr_cities = privateschools.groupby("City").sum()[["Total Enrollment"]].reset_index()
privateschools_enr_cities.sort_values("Total Enrollment", ascending=False)[:10]

,City,Total Enrollment
272,Los Angeles,37559
445,San Francisco,23964
448,San Jose,20526
442,San Diego,16040
433,Sacramento,12568
374,Pasadena,8158
346,Oakland,7212
462,Santa Ana,5046
349,Ojai,4770
422,Riverside,4676


In [216]:
enr15 = pd.read_table("publicschools15enr.txt")
enr15 = enr15.groupby("CDS_CODE").sum()[["ENR_TOTAL"]].reset_index()
pasadena_publics_enr = pd.merge(pasadena_publics, enr15, how='left', left_on="CDSCode", right_on="CDS_CODE")
pasadena_publics_enr

,CDSCode,School,CDS_CODE,ENR_TOTAL
0,19648811931062,Blair High,19648811931062,962
1,19648811931674,Marshall Fundamental,19648811931674,1971
2,19648811932409,Norma Coombs Elementary,19648811932409,458
3,19648811936103,John Muir High,19648811936103,902
4,19648811936806,Rose City High (Continuation),19648811936806,265
5,19648811936822,Pasadena High,19648811936822,1871
6,19648816021547,Cleveland Elementary,19648816021547,193
7,19648816021554,Don Benito Fundamental,19648816021554,608
8,19648816021570,Field (Eugene) Elementary,19648816021570,502
9,19648816021612,Hamilton Elementary,19648816021612,590


In [217]:
publicschools_enr = pd.merge(publicschools, enr15, how='left', left_on="CDSCode", right_on="CDS_CODE")
publicschools_enr_cities = publicschools_enr.groupby("CITY").sum()[["ENR_TOTAL"]].reset_index()
publicschools_enr_cities.sort_values("ENR_TOTAL", ascending=False)[:10]

,CITY,ENR_TOTAL
511,Los Angeles,320297
789,San Diego,180201
798,San Jose,153385
54,Bakersfield,118654
777,Sacramento,118613
318,Fresno,114321
886,Stockton,75829
505,Long Beach,68485
817,Santa Ana,67595
762,Riverside,62857


In [218]:
combined_enr_schools = pd.merge(publicschools_enr_cities, privateschools_enr_cities, how='outer', left_on="CITY", right_on="City")

def calculate_percentage_private(row):
    # return percentage private enrollment over total enrollment
    return row["Total Enrollment"] / (row["Total Enrollment"] + row["ENR_TOTAL"])

combined_enr_schools["private_pct"] = combined_enr_schools.apply(lambda x: calculate_percentage_private(x), axis=1)
combined_enr_schools["total_enr"] = combined_enr_schools.apply(lambda x: x["ENR_TOTAL"] + x["Total Enrollment"], axis=1)
combined_enr_schools.sort_values("private_pct", ascending=False)[:50]

,CITY,ENR_TOTAL,City,Total Enrollment,private_pct,total_enr
951,Valley Glen,70,Valley Glen,520,0.881356,590
28,Angwin,80,Angwin,216,0.729730,296
517,Los Olivos,172,Los Olivos,349,0.669866,521
500,Loma Linda,658,Loma Linda,1289,0.662044,1947
567,Mission Hills,736,Mission Hills,1333,0.644273,2069
635,Ojai,2637,Ojai,4770,0.643985,7407
510,Los Altos Hills,352,Los Altos Hills,546,0.608018,898
984,West Hollywood,478,West Hollywood,615,0.562672,1093
451,La Selva Beach,168,La Selva Beach,210,0.555556,378
890,Studio City,946,Studio City,1064,0.529353,2010
